<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51.jpg" style="float: right; padding: 70px 0px 0px 50px;" /></font>

# Combining MeerKAT with 2MASS

<p style="color:#c00000;">The ability to properly combine astronomical data at different wavelengths is a great multiplier that has really come into its own in the past couple of decades.  Whenever you do, you see new things, no matter how different the datasets seem at first.</p>

Here we are going to mosaic together near-infrared 2MASS data to make an exact match with a pre-made MeerKAT (1.28 GHz) radio map. The MeerKAT image is rich with hot bubbles of gas associated with new stars and supernova remnants whereas 2MASS shows dust clouds (or remnants of them) associated with star formation.


## Setup

The Montage Python package is a mixture of pure Python and Python binary extension code.  It can be downloaded using <tt style="color: #c00000;">pip install MontagePy</tt>.  You will also need OpenCV (<tt style="color: #c00000;">pip install opencv-python</tt>) for some cosmetic PNG manipulation at the end.


In [ ]:
# Startup.  The Montage modules are pretty much self-contained
# but this script needs a few extra utilities.  If you don't have
# these, they also can easily be "pip installed".

import os
import sys
import shutil
import cv2 as cv

from MontagePy.main    import *
from MontagePy.archive import *

from IPython.display import display, Image

import datetime
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))


# We create and move into several subdirectories in this notebook 
# but we want to come back to the original startup directory 
# whenever we restart the processing. 

# Here we use the directory where we started the Notebook as that
# home directory.  Feel free to explicitly point somewhere else.

try:
    home
except:
    home = os.getcwd()

os.chdir(home)

print("Startup folder: " + home)


## MeerKAT Data

The MeerKAT data we will use is already a radio mosaic.  With access to the raw observations, this mosaic could also be built using Montage but here we are going to leverage the work that others have done and just download the existing mosaic as our starting point:

https://archive-gw-1.kat.ac.za/public/repository/10.48479/fyst-hj47/data/MeerKAT_Galactic_Centre_1284MHz-StokesI.fits

We could perform the download from inside the Notebook but this is a big file (nearly a GByte) and you wouldn't want to download it every time you run this notebook.  So download it once however you prefer and save it to the directory designated as 'home' above.

### Extract the MeerKAT Image Header 

In order to build mosaics from 2MASS we need to define the header for the output mosaic FITS files.  And since we want our 2MASS mosaics to match the MeerKAT image, the simplest thing is to use the MeerKAP FITS header as the template for building them.  

The Montage mGetHdr tools extracts the header from an image and puts it into a text file.  That header can then be edited however we wish.  The MeerKAT header will contain a bunch of stuff specific to MeerKAT but since in this case we only care about projection information we can be lazy and just use it as is:

In [ ]:
# Extract the FITS header from the MeerKAT image:

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home)

meerkat = home + "/MeerKAT_Galactic_Centre_1284MHz-StokesI.fits"

rtn = mGetHdr(meerkat, "region.hdr")

print("mGetHdr:          " + str(rtn), flush=True)

rtn = mExamine(meerkat)

location = str(rtn["lonc"]) + " " + str(rtn["latc"])

size = rtn["ximgsize"]
if rtn["yimgsize"] > size:
    size = rtn["yimgsize"]
    
ratio = 1000./size

width  = rtn["ximgsize"] * ratio
height = rtn["yimgsize"] * ratio

shape = (int(width), int(height))

print("location:         " + location, flush=True)
print("size:             " + str(size))
print("shape:            " + str(shape))


## Working Environment

Before we get to actually building the mosaic, we need to set up our working environment.  Given the volume of data possible, the Montage processing is file based and we need to set up some subdirectories to hold bits of it.  This will all be under an instance-specific directory specified below ("workdir").  It is best not to use directory names with embedded spaces.

In [ ]:
# Clean out any old copy of the work tree, then remake it
# and the set of the subdirectories we will need.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

workdir = "2MASSJ"

os.chdir(home)

try:
    shutil.rmtree(workdir)
except:
    print("Can't delete work tree; probably doesn't exist yet.\n", flush=True)

print("Work directory: " + workdir, flush=True)

os.makedirs(workdir)  

os.chdir(workdir)

os.makedirs("raw")
os.makedirs("projected")
os.makedirs("diffs")
os.makedirs("corrected")


## Retrieving Data from an Archive

Now the first bit of Montage processing.  Montage uses standard FITS files throughout and FITS files have all the metadata describing the image (for us that mainly means pixel scale, projection type and center, rotation and so on).  To drive the processing we need a "FITS header" specification from the user, which we captured above in the header "template" file that looks just like a FITS header though with newlines (FITS headers have fixed 80-character card images with no line breaks).  Other ways to get a header template include mHdr, which creates a simple gnomonic projection header using center location, pixel scale and size) and mMakeHdr (which fits a bounding box around a set of images; usually the ones you are about to mosaic).  Or if you know enough you can just create one with an editor.

For data retrieval, Montage supports image metadata search using mSearch &mdash; a very fast R-Tree / memory-mapped utility &mdash; for most datasets.  This service returns URLs for all the images covering the region of interest, which are then downloaded.

There are many other ways to find images.  The International Virtual Astronomy Alliance (IVOA) has developed a couple of standards for querying metadata (Simple Image Access: SIAP and Table Access Protocol: TAP) which many data providers support.  Our service is focused on a few large uniform datasets (2MASS, DSS, SDSS, WISE).  Other datasets require more care.  For instance, simply downloading all pointed observations of a specific region for a non-survey instrument will include a wide range of integration times (and therefore noise levels) and the mosaicking should involve user-specified weighting of the images (which Montage supports but does not define).

In [ ]:
# Retrieve archive images covering the region then scan 
# the images for their coverage metadata.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

dataset = '2MASS J'

rtn = mArchiveDownload(dataset, location, size, "raw")

print("mArchiveDownload: " + str(rtn), flush=True)

rtn = mImgtbl("raw", "rimages.tbl")

print("mImgtbl (raw):    " + str(rtn), flush=True)

## Reprojecting the Images

In the last step above, we generated a list of all the images (with projection metadata) that had been successfully downloaded.  Using this and header template from above, we can now reproject all the images to a common frame.

Montage supplies four different reprojection modules to fit different needs.  mProject is completely general and is flux conserving but this results in it being fairly slow.  For a subset of projections (specifically where both the source and destination are tangent-plane projections) we can use a custom plane-to-plane algorithm developed by the Spitzer Space Telescope).  While mProjectPP only supports a subset of cases, they are extremely common ones.  mProjectPP is also flux conserving.

For fast reprojection, we relax the flux conservation requirement. However, even though we call attention to this explicitly in the name of the module: mProjectQL (quick-look), the results are indistinguishable from the other algorithms for all the tests we have run.

The fourth reprojection module, mProjectCube, is specifically for three- and four-dimensional datacubes.

mProjExec is a wrapper around the three main reprojection routines that determines whether mProjectPP or mProject should be used for each image (unless overridden with mProjectQL as here).  There is one final twist:  FITS headers allow for distortion parameters.  While these were introduced to deal with instrumental distortions, we can often use them to mimic an arbitrary projection over a small region with a distorted gnomonic projection.  This allows us to use mProjectPP over a wider range of cases and still have flux conservation with increased speed.

In [ ]:
# Reproject the original images to the  frame of the 
# output FITS header we created

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

rtn = mProjExec("raw", "rimages.tbl", "../region.hdr", projdir="projected", quickMode=True)

print("mProjExec:           " + str(rtn), flush=True)

mImgtbl("projected", "pimages.tbl")

print("mImgtbl (projected): " + str(rtn), flush=True)

## Coadding for a Mosaic

Now that we have a set of images all reprojected to a common frame, we can coadd them into a mosaic.  

In [ ]:
# Coadd the projected images without backgound correction.
# This step is just to illustrate the need for background correction
# and can be omitted.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

rtn = mAdd("projected", "pimages.tbl", "../region.hdr", "uncorrected.fits")

print("mAdd:    " + str(rtn), flush=True)

## View the Image

FITS files are binary data structures. To see the image we need to render to a JPEG or PNG form.  This involves choosing a stretch, color table (if it is a single image as here) and so on.  Montage provides a general visualization tool (mViewer) which can process a single image or multiple images for full color.  It supports overlays of various sorts.  One of its most useful features is a custom stretch algorithm which is based on gaussian-transformed histogram equalization, optionally with an extra logarithmic transform for really bright excursions.  A large fraction of astronomical images share the general characteristics of having a lot of pixels with something like a gaussian distribution at a low flux level (either background noise of low-level sky structure) coupled with a long histogram tail of very bright point-like sources.  If we apply our algorithm to this, stretching from the -2 or -1 "sigma" value of the low-level distribution to the image brightness maximum we usually get a good balance of seeing the low-level structure while still seeing the structure and brightness variations of the bright sources.

mViewer specifications can become quite lengthy so the module provides three entry mechanisms.  The most terse (used here) is a "parameter string" based on the command-line arguments of the original stand-alone C program.  For more complicated descriptions the user can define a JSON string or JSON file.  See the <a style="text-decoration: none; color: #c00000" href="https://github.com/Caltech-IPAC/MontageNotebooks/blob/main/mViewer.ipynb"> Sky Visualization </a> notebook example.

We use the built-in IPython.display utility to show the resultant image, which shrinks it to fit.  There are several other tools you can use instead.

In [ ]:
# Make a PNG rendering of the data and display it.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

rtn = mViewer("-ct 1 -gray uncorrected.fits -2s max gaussian-log -out uncorrected.png", "", mode=2)

print("mViewer: " + str(rtn), flush=True)

# Jupyter IPython display gets unhappy with big images, so shrink the PNG

img = cv.imread("uncorrected.png")
img = cv.resize(img, shape)
cv.imwrite('uncorrected_small.png', img)

Image(filename='uncorrected_small.png')

## Background Matching

We can do better.  In the above image there are vertical stripes.  Even though the images were accurately flux-calibrated the background levels in the individual image varied due to real changes in the brightness of the sky (2MASS data was taken from the ground, so the atmosphere was a variable).

This is a common problem; differential photometry is easier than absolute.  So Montage provides tools for determining what is essentially a compromise background:  Not flattened (since in the above mosaic there is real structure throughout the image) and not modelled (there might be a model you can develop for the Galactic structure above but it wouldn't do that good a job of cleaning up the mosaic). 

Rather, we ask what is the set of minimum adjustments we can make to the individual image backgrounds to bring them all in-line with each others.  Often, this is just a constant offset; at most it includes slight slopes.  Anything more and we are starting to fit the sky structure rather than the background differences.

The first steps is determining the corrections is to analyze the overlap areas between adjacent images.  We determine from the image metadata (which again includes sky coverage) where there are overlaps.  Then for each pairwise overlap, we compute the image difference.  There is an explicit assumption here that a such a pair the sources and other real-sky structure match (including flux scales) so the difference should have nothing in it but background differences.  We then fit each difference with a plane (ignoring large excursions just to be safe).

Finally, given this set of difference fits, we determine iteratively a global mimimum difference which results in a set of corrections to each image.

In [ ]:
# Determine the overlaps between images (for background modeling).

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

rtn = mOverlaps("pimages.tbl", "diffs.tbl")

print("mOverlaps:    " + str(rtn), flush=True)


# Generate difference images and fit them.

rtn = mDiffFitExec("projected", "diffs.tbl", "../region.hdr", "diffs", "fits.tbl")

print("mDiffFitExec: " + str(rtn), flush=True)


# Model the background corrections.

rtn = mBgModel("pimages.tbl", "fits.tbl", "corrections.tbl")

print("mBgModel:     " + str(rtn), flush=True)

## Background Correcting and Re-Mosaicking

Now all we have to do is apply the background corrections to the individual images and regenerate the mosaic.  While we don't attempt to maintain the global total flux (this would be meaningless in any case given the source of the offsets), in general our final mosaic is close to this level.

For those cases where the background should truly be flat (extragalactic fields with no foreground we want to keep) Montage also provides simple "flattening" tools.

In [ ]:
# Background correct the projected images.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

rtn = mBgExec("projected", "pimages.tbl", "corrections.tbl", "corrected")

print("mBgExec:             " + str(rtn), flush=True)

rtn = mImgtbl("corrected", "cimages.tbl")

print("mImgtbl (corrected): " + str(rtn), flush=True)


# Coadd the background-corrected, projected images.

rtn = mAdd("corrected", "cimages.tbl", "../region.hdr", "2massj.fits")

print("mAdd:                " + str(rtn), flush=True)

## Final Image

Now when we regenerate and display a PNG for the mosaic, it has no artifacts and all of the low-level structure is preserved.

In [ ]:
# Make a PNG rendering of the data and display it.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSJ")

rtn = mViewer("-ct 1 -gray 2massj.fits -2s max gaussian-log -out 2massj.png", "", mode=2)

img = cv.imread("2massj.png")
img = cv.resize(img, shape)
cv.imwrite('2massj_small.png', img)

Image(filename='2massj_small.png')

## Full Color and Overlays

The above can be packaged up in a Python script with whatever minimum input and defaults you desired.  Repeat the processing for three different wavelengths and you can combine them (and optionally overlays of various sorts) into a full-color image. Below, in compressed form, are the other two 2MASS wavelengths.

For clarity, we've removed error checking and debugging output.



## H-Band

In [ ]:
# Clean out any old copy of the work tree for 2MASS H, then remake it
# and the set of the subdirectories we will need.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

workdir = "2MASSH"

os.chdir(home)

try:
    shutil.rmtree(workdir)
except:
    print("Can't delete work tree; probably doesn't exist yet.\n", flush=True)

os.makedirs(workdir)  

os.chdir(workdir)

os.makedirs("raw")
os.makedirs("projected")
os.makedirs("diffs")
os.makedirs("corrected")

In [ ]:
dataset = "2MASS H"

os.chdir(home + "/2MASSH")

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

mArchiveDownload(dataset, location, size, "raw")
rtn = mImgtbl("raw", "rimages.tbl")

print("Raw images:          " + str(rtn), flush=True)

In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSH")

mProjExec("raw", "rimages.tbl", "../region.hdr", projdir="projected", quickMode=True)
mImgtbl("projected", "pimages.tbl")

mOverlaps("pimages.tbl", "diffs.tbl")

mDiffFitExec("projected", "diffs.tbl", "../region.hdr", "diffs", "fits.tbl")

mBgModel("pimages.tbl", "fits.tbl", "corrections.tbl", "")

mBgExec("projected", "pimages.tbl", "corrections.tbl", "corrected")
mImgtbl("corrected", "cimages.tbl")

rtn = mAdd("corrected", "cimages.tbl", "../region.hdr", "2massh.fits")

mViewer("-ct 1 -gray 2massh.fits -2s max gaussian-log -out 2massh.png", "", mode=2)

img = cv.imread("2massh.png")
img = cv.resize(img, shape)
cv.imwrite('2massh_small.png', img)

Image(filename='2massh_small.png')



## K-Band

In [ ]:
# Clean out any old copy of the work tree for 2MASS H, then remake it
# and the set of the subdirectories we will need.

now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

workdir = "2MASSK"

os.chdir(home)

try:
    shutil.rmtree(workdir)
except:
    print("Can't delete work tree; probably doesn't exist yet.\n", flush=True)

os.makedirs(workdir)  

os.chdir(workdir)

os.makedirs("raw")
os.makedirs("projected")
os.makedirs("diffs")
os.makedirs("corrected")

In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

dataset = "2MASS K"

mArchiveDownload(dataset, location, size, "raw")
rtn = mImgtbl("raw", "rimages.tbl")

print("Raw images:          " + str(rtn), flush=True)

In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSK")

mProjExec("raw", "rimages.tbl", "../region.hdr", projdir="projected", quickMode=True)
mImgtbl("projected", "pimages.tbl")

mOverlaps("pimages.tbl", "diffs.tbl")

mDiffFitExec("projected", "diffs.tbl", "../region.hdr", "diffs", "fits.tbl")

mBgModel("pimages.tbl", "fits.tbl", "corrections.tbl", "")

mBgExec("projected", "pimages.tbl", "corrections.tbl", "corrected")
mImgtbl("corrected", "cimages.tbl")

rtn = mAdd("corrected", "cimages.tbl", "../region.hdr", "2massk.fits")

mViewer("-ct 1 -gray 2massk.fits -2s max gaussian-log -out 2massk.png", "", mode=2)

img = cv.imread("2massk.png")
img = cv.resize(img, shape)
cv.imwrite('2massk_small.png', img)

Image(filename='2massk_small.png')



## Create 2MASS Color Mosaic

Now that we have FITS images for all three (J, H and K) 2MASS wavelengths, we can create a color-composite 2MASS PNG for the region:

In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home + "/2MASSK")

os.chdir(home)

rtn = mViewer("-blue  2MASSJ/mosaic.fits -1s max gaussian-log "
              "-green 2MASSH/mosaic.fits -1s max gaussian-log "
              "-red   2MASSK/mosaic.fits -1s max gaussian-log -out 2MASS.png",
              "", mode=2)

img = cv.imread("2MASS.png")
img = cv.resize(img, shape)
cv.imwrite('2MASS_small.png', img)

Image('2MASS_small.png')


and a grayscale image with exactly the same coverage from MeerKAT:

In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

os.chdir(home)

rtn = mViewer("-gray MeerKAT_Galactic_Centre_1284MHz-StokesI.fits "
              "-1s max gaussian-log -out MeerKAT.png",
              "", mode=2)

img = cv.imread("MeerKAT.png")
img = cv.resize(img, shape)
cv.imwrite('MeerKAT_small.png', img)

Image('MeerKAT_small.png')



## Overlay 2MASS and MeerKAT Images and Blend Together

There are several options for combining the MeerKAT and 2MASS.  We could build a 3-color composite using two of the 2MASS wavelengths and MeerKAT (probably 2MASS J, 2MASS K, and MeerKAT) but we have chosen instead to use "photo-processing") toolkit the blend the two images above together.  The MeerKAT image is here overlaid as a grayscale, translucent layer on the full color 2MASS image.  There are a lot of options to how we set the brightness and contrast of each and the relative weight of the blending.  Feel free to tweak these parameters to emphasize the features you wish to illustrate.

In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

contrast   = 2.0
brightness = -1.0

os.chdir(home)

tmassPath = os.path.join(home, '2MASS.png')
tmass = cv.imread(tmassPath)

tmassStretch = cv.addWeighted(tmass, contrast, tmass, 0, brightness)

cv.imwrite('2massStretch.png', tmassStretch)

# -------------------
   
contrast   = 3.0
brightness = -128.0

meerkatPath = os.path.join(home, 'MeerKAT.png')
meerkat = cv.imread(meerkatPath)

meerkatStretch = cv.addWeighted(meerkat, contrast, meerkat, 0, brightness)

cv.imwrite('meerkatStretch.png', meerkatStretch)

# -------------------

alpha = 0.6
beta  = 1.0 - alpha
gamma = 0.0

imgBlend = cv.addWeighted(tmassStretch, alpha, meerkatStretch, beta, gamma)

cv.imwrite('MeerKAT_2MASS.png', imgBlend)

img = cv.imread("MeerKAT_2MASS.png")
img = cv.resize(img, shape)
cv.imwrite('MeerKAT_2MASS_small.png', img)

Image(filename="MeerKAT_2MASS_small.png")


Note that all of the images shown on the page are essentially thumbnails; they have been shrunken by a factor of about 10 in both dimensions.

Also, except for the final composite above, no attempt has been made to perform brightness stretching on the images.  See the files on disk for the original data.


In [ ]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))